<a href="https://colab.research.google.com/github/AkerkeKesha/stacked-unets/blob/main/gcolab_spacenet6_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up plotting options
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
gdrive_root = '/content/drive/MyDrive/stacked-unet/dataset/data-spacenet6'
gdrive_root

'/content/drive/MyDrive/stacked-unet/dataset/data-spacenet6'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!git clone https://github.com/AkerkeKesha/stacked-unets.git

Cloning into 'stacked-unets'...
remote: Enumerating objects: 1129, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 1129 (delta 154), reused 179 (delta 94), pack-reused 880
Receiving objects: 100% (1129/1129), 6.46 MiB | 18.01 MiB/s, done.
Resolving deltas: 100% (758/758), done.


In [ ]:
%cd stacked-unets

/content/stacked-unets


In [ ]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=e71ca118377fdd9a224774870fda7a539999c307f0d3a505e3479e9f5d264e5b
  Stored in direct

In [ ]:
!mkdir -p /content/stacked-unets/dataset/data-spacenet6

!tar -xzf $gdrive_root/SN6_buildings_AOI_11_Rotterdam_train.tar.gz -C /content/stacked-unets/dataset/data-spacenet6

In [ ]:
%pwd

In [ ]:
!mkdir -p dataset/data-spacenet6/train/AOI_11_Rotterdam
!mv dataset/data-spacenet6/SAR-Intensity dataset/data-spacenet6/train/AOI_11_Rotterdam/
!mv dataset/data-spacenet6/SummaryData dataset/data-spacenet6/train/AOI_11_Rotterdam/
!mv dataset/data-spacenet6/masks dataset/data-spacenet6/train/AOI_11_Rotterdam/
!mv dataset/data-spacenet6/geojson_buildings dataset/data-spacenet6/train/AOI_11_Rotterdam/

mv: cannot move 'dataset/data-spacenet6/SAR-Intensity' to 'dataset/data-spacenet6/train/AOI_11_Rotterdam/SAR-Intensity': Directory not empty
mv: cannot move 'dataset/data-spacenet6/SummaryData' to 'dataset/data-spacenet6/train/AOI_11_Rotterdam/SummaryData': Directory not empty
mv: cannot move 'dataset/data-spacenet6/masks' to 'dataset/data-spacenet6/train/AOI_11_Rotterdam/masks': Directory not empty
mv: cannot move 'dataset/data-spacenet6/geojson_buildings' to 'dataset/data-spacenet6/train/AOI_11_Rotterdam/geojson_buildings': Directory not empty


# Couple check ups

In [ ]:
%pwd

In [ ]:
!git fetch

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 501 bytes | 501.00 KiB/s, done.
From https://github.com/AkerkeKesha/stacked-unets
   3895877..ba778fc  main       -> origin/main


In [ ]:
!git pull

Updating 3895877..ba778fc
Fast-forward
 src/dataset.py | 12 +++++++-----
 1 file changed, 7 insertions(+), 5 deletions(-)


In [ ]:
!git stash

Saved working directory and index state WIP on main: 306874a added distortions per level


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available


In [ ]:
import sys

sys.path.append('/content/stacked-unets/src')

In [ ]:
import os

os.environ['ENVIRONMENT'] = 'colab'
os.environ['DATASET'] = 'sn6'
os.environ['STOP'] = 'no'

In [ ]:
import os
from glob import glob
import config

print(f"Is colab: {config.environment}")
print(f"Is colab: {config.dataset}")
print(f"Training dir: {config.train_dir}")
print(f"{config.stop}")

Is colab: colab
Is colab: sn6
Training dir: /content/stacked-unets/dataset/data-spacenet6/train/AOI_11_Rotterdam
no


In [5]:
from src.main import start_stacked_unet

start_stacked_unet(n_levels=1, max_data_points=1000)

Split into train:(800, 4), validation:(100, 4), and test:(100, 4)
Level: [1 / 1]


  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

An exception occurred during validation: CUDA out of memory. Tried to allocate 2.46 GiB (GPU 0; 14.75 GiB total capacity; 5.55 GiB already allocated; 2.31 GiB free; 10.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

An exception occurred during validation: CUDA out of memory. Tried to allocate 2.46 GiB (GPU 0; 14.75 GiB total capacity; 5.55 GiB already allocated; 720.81 MiB free; 12.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


  0%|          | 0/17 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
!zip -r output_baseline_sn6.zip output > /dev/null 2>&1


In [ ]:
!cp output_baseline_sn6.zip "/content/drive/MyDrive/stacked-unet/output_baseline_sn6.zip"

In [ ]:
!ls output/sn6_labels | wc -l